# 标的列表获取

In [5]:
import tushare as ts
import pandas as pd
import time
import os
from tqdm import tqdm

def get_stock_data():
    # 读取中证500成分股数据
    # 数据文件路径
    file_path = r"D:\read\task\中证500成分股,单一股票数据.csv"
    # 读取CSV文件，注意处理带引号的字段
    df = pd.read_csv(file_path, dtype=str)  # 先都按字符串读取，避免格式问题
    # 获取股票代码列表
    stock_codes = df['con_code'].unique()
    print (stock_codes)
    
if __name__ == "__main__":
    get_stock_data()

['300476.SZ' '000988.SZ' '300450.SZ' '688521.SH' '300207.SZ' '600580.SH'
 '300803.SZ' '002156.SZ' '600988.SH' '300339.SZ' '002558.SZ' '002195.SZ'
 '600487.SH' '002340.SZ' '688072.SH' '600143.SH' '688525.SH' '600021.SH'
 '002517.SZ' '002837.SZ' '002738.SZ' '601168.SH' '000831.SZ' '300857.SZ'
 '000933.SZ' '601555.SH' '000426.SZ' '603893.SH' '600839.SH' '002532.SZ'
 '601162.SH' '002966.SZ' '300604.SZ' '600096.SH' '002436.SZ' '300115.SZ'
 '688027.SH' '600157.SH' '600885.SH' '688122.SH' '689009.SH' '002273.SZ'
 '002008.SZ' '300136.SZ' '002850.SZ' '600895.SH' '002261.SZ' '688235.SH'
 '600549.SH' '688120.SH' '002851.SZ' '002294.SZ' '301358.SZ' '300454.SZ'
 '688777.SH' '301308.SZ' '300395.SZ' '000066.SZ' '600392.SH' '002202.SZ'
 '002797.SZ' '600673.SH' '688469.SH' '600699.SH' '603659.SH' '002185.SZ'
 '002138.SZ' '601099.SH' '002353.SZ' '002281.SZ' '688002.SH' '300073.SZ'
 '688213.SH' '000021.SZ' '002472.SZ' '600536.SH' '300223.SZ' '000783.SZ'
 '688099.SH' '600879.SH' '688608.SH' '603228.SH' '3

# 历史数据获取函数

In [7]:
import os
import pandas as pd

def get_price(security, start_date=None, end_date=None, frequency='daily', fields=None,
              skip_paused=False, count=None, panel=True, fill_paused=True):
    """
    获取历史数据，可查询多个标的多个数据字段，返回数据格式为 DataFrame
    """
    # 数据文件路径
    file_path = r"D:\read\task\机器学习数据.pkl"

    # 检查文件是否存在
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"数据文件不存在: {file_path}")

    # 读取pkl文件
    df = pd.read_pickle(file_path)

    # 转换数值字段为适当类型
    numeric_fields = ['open', 'high', 'low', 'close', 'pre_close', 
                      'change', 'pct_chg', 'vol', 'amount']
    for field in numeric_fields:
        if field in df.columns:
            df[field] = pd.to_numeric(df[field], errors='ignore')

    # 转换日期列的格式
    if 'trade_date' in df.columns:
        # 假设日期是整数格式如20180102，转换为datetime
        df['trade_date'] = pd.to_datetime(df['trade_date'], format='%Y%m%d')

    # 处理股票代码
    if 'ts_code' in df.columns:
        # 确保股票代码是字符串格式
        df['ts_code'] = df['ts_code'].astype(str)

    # 过滤股票代码
    if isinstance(security, list):
        # 多个股票代码
        securities = [str(s) for s in security]
        df = df[df['ts_code'].isin(securities)]
    else:
        # 单个股票代码
        security_str = str(security)
        df = df[df['ts_code'] == security_str]

    # 过滤日期范围
    if start_date:
        start_date = pd.to_datetime(start_date)
        df = df[df['trade_date'] >= start_date]

    if end_date:
        end_date = pd.to_datetime(end_date)
        df = df[df['trade_date'] <= end_date]

    # 选择需要的字段
    available_fields = ['ts_code', 'trade_date', 'open', 'high', 'low', 'close',
                        'pre_close', 'change', 'pct_chg', 'vol', 'amount']

    if fields:
        # 检查字段是否有效
        invalid_fields = [f for f in fields if f not in available_fields]
        if invalid_fields:
            raise ValueError(f"无效的字段: {invalid_fields}，可用字段: {available_fields}")

        # 确保保留股票代码和日期
        selected_fields = fields.copy()
        if 'ts_code' not in selected_fields:
            selected_fields.insert(0, 'ts_code')
        if 'trade_date' not in selected_fields:
            selected_fields.insert(1, 'trade_date')

        df = df[selected_fields]

    # 按日期排序
    if 'trade_date' in df.columns:
        df = df.sort_values('trade_date')

    # 限制返回的记录数量
    if count and count > 0:
        df = df.tail(count)

    return df

if __name__ == "__main__":
    # 示例用法
    data = get_price(security=['300450.SZ'], start_date='2020-01-01', end_date='2020-12-31', fields=['open', 'close', 'vol'])
    print(data)


C:\Users\chanpi\AppData\Local\Temp\ipykernel_26556\4110505164.py:24: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[field] = pd.to_numeric(df[field], errors='ignore')


          ts_code trade_date       open      close        vol
396501  300450.SZ 2020-01-02  46.054590  46.085036  160696.31
396502  300450.SZ 2020-01-03  46.074887  46.267711  121498.40
396503  300450.SZ 2020-01-06  46.967968  46.947671  223693.90
396504  300450.SZ 2020-01-07  46.947671  46.288009  146123.26
396505  300450.SZ 2020-01-08  46.267711  45.465968  132403.35
...           ...        ...        ...        ...        ...
396739  300450.SZ 2020-12-25  70.651882  72.084745   97963.68
396740  300450.SZ 2020-12-28  72.801176  74.616789  162863.44
396741  300450.SZ 2020-12-29  74.322365  72.428239  135546.35
396742  300450.SZ 2020-12-30  76.638498  83.410244  318000.10
396743  300450.SZ 2020-12-31  82.428832  82.428832  189888.70

[243 rows x 5 columns]


ModuleNotFoundError: No module named 'Data_Handling'